## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)

def canny(img, th, tl):
    X, Y = img.shape
    img_blur = cv2.GaussianBlur(img ,(5, 5), 0)
    S1=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    S2=np.array([[-1,-2,-1],[0,0,0],[1,2,1]])
    Gx=cv2.filter2D(img_blur, -1, S1)
    Gy=cv2.filter2D(img_blur, -1, S2)
    ampli = np.sqrt(Gx**2 + Gy**2)
    alfa = np.arctan2(Gx, Gy)
    alfa_deg = alfa * 180/np.pi
    alfa_deg[alfa_deg < 0] += 180
    kier = np.zeros((X, Y))

    for i in range(X):
        for j in range(Y):
            if (0 <= alfa_deg[i,j] < 22.5) or (157.5 <= alfa_deg[i,j] <= 180):
                kier[i,j]=1
            elif (22.5 <= alfa_deg[i,j] < 67.5):
                kier[i,j]=2
            elif (67.5 <= alfa_deg[i,j] < 112.5):
                kier[i,j]=3
            elif (112.5 <= alfa_deg[i,j] < 157.5):
                kier[i,j]=4
    gn = nonmax(kier, ampli)
    gnh = gn >= th
    gnl=np.where(np.logical_and(th > gn, gn >= tl), 1, 0)
    return gnh, gnl

def nonmax(kier,ampli):
    X, Y = kier.shape
    gn = np.zeros((X, Y))
    
    for i in range (1, X - 1):
        for j in range (1, Y - 1):
            if kier[i, j] == 1:
                if ampli[i, j - 1] > ampli[i, j] or ampli[i, j + 1] > ampli[i, j]:
                    gn[i, j] = 0
                else:
                    gn[i, j] = ampli[i, j]
            elif kier[i, j] == 2:
                if ampli[i + 1, j - 1] > ampli[i, j] or ampli[i - 1, j + 1] > ampli[i, j]:
                    gn[i, j] = 0
                else:
                    gn[i, j] = ampli[i, j]
            elif kier[i, j] == 3:
                if ampli[i + 1, j] > ampli[i, j] or ampli[i - 1, j] > ampli [i, j]:
                    gn[i,j] = 0
                else:
                    gn[i, j] = ampli[i, j]
            elif kier[i, j] == 4:
                if ampli[i - 1, j - 1] > ampli[i, j] or ampli[i + 1, j + 1] > ampli[i, j]:
                    gn[i, j] = 0
                else:
                    gn[i, j] = ampli[i, j]
    return gn

gnh, gnl = canny(house, tl = 5, th = 10)

plt.imshow(gnh, 'gray', vmin = 0, vmax = 1)
plt.axis('off')
plt.show()

plt.imshow(gnl, 'gray', vmin = 0, vmax = 1)
plt.axis('off')
plt.show()

In [ ]:
def canny_2(gnh, gnl, img):
    X, Y = img.shape
    lista = []
    for i in range(1, X - 1):
        for j in range(1, Y - 1):
            if gnh[i, j] == 1:
                lista.append([i, j])
    visited = np.zeros((X, Y))
    edges  = np.zeros((X, Y))
    while(lista):
        el = lista.pop()
        x = el[0]
        y = el[1]
        if visited[x, y] == 0 :
            visited[x, y] = 1
            edges[x, y] = 1
            for ii in range(x - 1, x + 2):
                for jj in range(y - 1, y + 2):
                    if gnl[ii, jj] == 1:
                        lista.append([ii, jj])
    return edges
gnh,gnl = canny(house,tl=5,th=10)

img_canny = canny_2(gnh = gnh, gnl = gnl,img = house)
img_can=cv2.Canny(house,5,10,None,3,1)

f, (ax1, ax2) = plt.subplots(2, 2, figsize = (8, 8))
ax1[0].set_title("Oryginał")
ax1[0].imshow(house, 'gray')
ax1[0].axis('off')
ax1[1].set_title("GNH")
ax1[1].imshow(gnh, 'gray')
ax1[1].axis('off')
ax2[0].set_title("Wynik")
ax2[0].imshow(img_canny, 'gray')
ax2[0].axis('off')
ax2[1].set_title("Z biblioteki")
ax2[1].imshow(img_can, 'gray')
ax2[1].axis('off')